# Practica 2 - Natural Language processing

In [13]:
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import Model, Input, layers
from tensorflow.keras.layers import Embedding, Dot, Reshape, Dense, TextVectorization

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams

from keras.callbacks import TensorBoard

import seaborn as sns
import os

In [2]:
df_train = pd.read_csv(os.path.join("data", "train.csv"), sep=',', header=0, encoding='ISO-8859-1', index_col="textID")
df_test = pd.read_csv(os.path.join("data", "test.csv"), sep=',', header=0, encoding='ISO-8859-1', index_col="textID")

Comprobamos si hay NAs en el dataset: 

In [3]:
df_train.isna().sum()

text                1
selected_text       1
sentiment           0
Time of Tweet       0
Age of User         0
Country             0
Population -2020    0
Land Area (Km²)     0
Density (P/Km²)     0
dtype: int64

Podemos observar que hay uno:

In [4]:
df_train[df_train.isna().any(axis = 1)]

,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
textID,,,,,,,,,
fdb77c3752,NaN,NaN,neutral,night,31-45,Namibia,2540905,823000.0,3


In [5]:
df_train = df_train.dropna()
df_train.isna().sum()

text                0
selected_text       0
sentiment           0
Time of Tweet       0
Age of User         0
Country             0
Population -2020    0
Land Area (Km²)     0
Density (P/Km²)     0
dtype: int64

In [6]:
df_test.isna().sum()

text                1281
sentiment           1281
Time of Tweet       1281
Age of User         1281
Country             1281
Population -2020    1281
Land Area (Km²)     1281
Density (P/Km²)     1281
dtype: int64

In [7]:
df_test[df_test.isna().any(axis = 1)].head(5)

,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
textID,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_test = df_test.dropna()
df_test.isna().sum()

text                0
sentiment           0
Time of Tweet       0
Age of User         0
Country             0
Population -2020    0
Land Area (Km²)     0
Density (P/Km²)     0
dtype: int64

In [9]:
corpus = df_train['text'].values

In [10]:
vectorize_layer = TextVectorization(
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    output_mode='int'
)

In [11]:
vectorize_layer.adapt(corpus)

In [12]:
vectorized_train = vectorize_layer(df_train["text"].values)

In [13]:
vectorized_train

<tf.Tensor: shape=(27480, 33), dtype=int64, numpy=
array([[  293,    17, 15185, ...,     0,     0,     0],
       [  413,   115,     2, ...,     0,     0,     0],
       [    6,  1335,    10, ...,     0,     0,     0],
       ...,
       [  225,    31,    12, ...,     0,     0,     0],
       [   20,     9,    28, ...,     0,     0,     0],
       [   29,    30,  6480, ...,     0,     0,     0]])>

In [21]:
vocab = vectorize_layer.get_vocabulary()

In [27]:
token_to_index = {word: idx for idx, word in enumerate(vocab)}
index_to_token = {idx: word for idx, word in enumerate(vocab)}

In [52]:
text_ds = tf.data.Dataset.from_tensor_slices(df_train["text"].values)

In [54]:
vectorized_ds = text_ds.map(lambda x: vectorize_layer(x))

In [ ]:
vocab = vectorize_layer.get_vocabulary()
vocab_size = len(vocab)

def tf_skipgrams(sequence):
    sequence = sequence.numpy()
    couples, labels = skipgrams(
        sequence=sequence,
        vocabulary_size=vocab_size,
        window_size=4,
    )
    if len(couples) == 0:
        return (tf.constant([[0, 0]], dtype=tf.int32), tf.constant([0], dtype=tf.int32))
    return (tf.constant(couples, dtype=tf.int32), tf.constant(labels, dtype=tf.int32))

def tf_skipgrams_wrapper(sequence):
    couples, labels = tf.py_function(tf_skipgrams, [sequence], [tf.int32, tf.int32])
    couples.set_shape([None, 2])
    labels.set_shape([None])
    return couples, labels

skipgram_ds = vectorized_ds.map(tf_skipgrams_wrapper)

In [ ]:
import os
import pandas as pd

data_dir = "data"
train_path = os.path.join(data_dir, "train.csv")
test_path = os.path.join(data_dir, "test.csv")
df_train = pd.read_csv(train_path, encoding='ISO-8859-1', index_col="textID")
df_test = pd.read_csv(test_path, encoding='ISO-8859-1', index_col="textID")
df_train.head()

,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Kmï¿½),Density (P/Kmï¿½)
textID,,,,,,,,,
cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [14]:
df_train.isna().sum()

text                 1
selected_text        1
sentiment            0
Time of Tweet        0
Age of User          0
Country              0
Population -2020     0
Land Area (Kmï¿½)    0
Density (P/Kmï¿½)    0
dtype: int64

In [15]:
df_test.isna().sum()

text                1281
sentiment           1281
Time of Tweet       1281
Age of User         1281
Country             1281
Population -2020    1281
Land Area (Km²)     1281
Density (P/Km²)     1281
dtype: int64

In [16]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [17]:
df_train["text"].isna().sum()

np.int64(0)

In [54]:
from tensorflow.keras.layers import TextVectorization

vectorize_layer = TextVectorization(
    output_mode='int',
    standardize="lower_and_strip_punctuation",
    split="whitespace",
)

corpus = df_train["text"].values

vectorize_layer.adapt(corpus)

vectorized_train = vectorize_layer(corpus)
vectorized_train

<tf.Tensor: shape=(27480, 33), dtype=int64, numpy=
array([[  293,    17, 15185, ...,     0,     0,     0],
       [  413,   115,     2, ...,     0,     0,     0],
       [    6,  1335,    10, ...,     0,     0,     0],
       ...,
       [  225,    31,    12, ...,     0,     0,     0],
       [   20,     9,    28, ...,     0,     0,     0],
       [   29,    30,  6480, ...,     0,     0,     0]])>

In [55]:
vectorize_layer.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('i'),
 np.str_('to'),
 np.str_('the'),
 np.str_('a'),
 np.str_('my'),
 np.str_('and'),
 np.str_('you'),
 np.str_('it')]

In [56]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import skipgrams

train_ds = tf.data.Dataset.from_tensor_slices(vectorized_train)

vocab_size = len(vectorize_layer.get_vocabulary())
window_size = 3

def tf_skipgrams(sequence):
    sequence = tf.cast(sequence, tf.int32)
    
    def generate_pairs(x):
        pairs, _ = skipgrams(
            x.numpy(),
            vocabulary_size=vocab_size,
            window_size=window_size,
            negative_samples=0  # set to >0 if you want (target, context), label
        )
        if not pairs:
            # Return dummy data if skipgrams returns empty
            return tf.zeros((0, 2), dtype=tf.int32)
        return tf.convert_to_tensor(pairs, dtype=tf.int32)

    pairs_tensor = tf.py_function(func=generate_pairs, inp=[sequence], Tout=tf.int32)
    return tf.data.Dataset.from_tensor_slices(pairs_tensor)

# Use flat_map to flatten pairs
train_ds = train_ds.flat_map(tf_skipgrams)

# Shuffle and batch
next(train_ds.as_numpy_iterator())

array([69, 47], dtype=int32)

In [59]:
train_ds = (
    train_ds
    .map(lambda x: (tf.one_hot(x[0], depth=vocab_size), tf.one_hot(x[1], depth=vocab_size)))
)
next(train_ds.as_numpy_iterator())

(array([0., 0., 1., ..., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], dtype=float32))